### Unsupervised Learning




### ML EXAM 2

### UNSUPERVISED LEARNING

## import the required libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('credit_card.csv')
df.shape

In [ ]:
# After reading the dataset, let’s check its top 5 rows using head() method to just see how the data looks.
df.head()

In [ ]:
df.dtypes

###  **Q.1 Basic Primary Analysis**

---


In [ ]:
sns.set_style("darkgrid")
plt.figure(figsize=(7,7))

In [ ]:
sns.scatterplot(x='BALANCE',y='PURCHASES_FREQUENCY',data=df)

In [ ]:
df['CUST_ID'].describe()

###  **Q.2 Basic Exploratory Analysis**

---

**a. Missing Value Analysis**

In [ ]:
df.isnull().sum()

In [ ]:
# to see what percentage of the values in the dataset were missing
total = np.product(df.shape)
count = df.isnull().sum()
total_missing =count.sum()

# percent of data that is missing
(total_missing/total)*100

***Only 0.19% percent of data are missing from the dataset***

**Handling Missing Values**

In [ ]:
# The column 'MINIMUM_PAYMENTS'  has 313 missing values, since it is a numerical variable the missing values in that column can be filled with mean 
#replacing  'MINIMUM_PAYMENTS'  column with mean value
mean_value=df['MINIMUM_PAYMENTS'].mean()
df['MINIMUM_PAYMENTS'].fillna(value=mean_value,inplace=True)

In [ ]:
mean_value=df['CREDIT_LIMIT'].mean()
df['CREDIT_LIMIT'].fillna(value=mean_value,inplace=True)

**b. Outlier Treatment with using Z-score method**

In [ ]:
sns.boxplot(df.PURCHASES)
plt.show()

In [ ]:
f, ax = plt.subplots(3,3, figsize=(20,20))
fig1= sns.distplot(df["PURCHASES_INSTALLMENTS_FREQUENCY"],bins=10, ax= ax[0,0])
fig2 = sns.distplot(df["CASH_ADVANCE_FREQUENCY"],bins=10, ax=ax[0,1])
fig3 = sns.distplot(df["CASH_ADVANCE_TRX"],bins=10, ax= ax[0,2])
fig4 = sns.distplot(df["PURCHASES_TRX"],bins=10, ax=ax[1,0])
fig5 = sns.distplot(df["CREDIT_LIMIT"],bins=10, ax= ax[1,1])
fig6 = sns.distplot(df["PAYMENTS"],bins=10, ax=ax[1,2])
fig7 = sns.distplot(df["MINIMUM_PAYMENTS"],bins=10, ax= ax[2,0])
fig8 = sns.distplot(df["PRC_FULL_PAYMENT"],bins=10, ax=ax[2,1])
fig9 = sns.distplot(df["TENURE"],bins=10, ax=ax[2,2])


In [ ]:
from scipy.stats import zscore     # To Apply Zscore Treatment

z = np.abs(zscore(df['PURCHASES'])) 

In [ ]:
df['Zscore'] = z

len(df[df['Zscore']>2])

In [ ]:
df_z = df[df['Zscore']<3]      # Store less then 3 Z-Score value To the Variable
print('Removed outlier data set shape :-',df_z.shape  ) 

In [ ]:
df.shape

In [ ]:
df_z.shape # the outliers of the dataset has been dropped

**c. Dealing with correlated variables**





In [ ]:
df.drop('Zscore',axis=1,inplace=True)
df.corr()

In [ ]:
f, ax = plt.subplots(figsize=(15,10 ))
sns.heatmap(df.corr(),annot=True,cmap="YlGnBu")

***The Atrribute ONEOFF_PURCHASES and PURCHASES are highly correlated.hence one of the column is removed***

In [ ]:
df.drop('ONEOFF_PURCHASES',axis=1,inplace=True) # highly correlated attribute has been removed.

###  **Q.3 Dimensionality Reduction using PCA**

---

In [ ]:
df_pca=df.copy()
df_pca.drop('CUST_ID',axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

df_st =  StandardScaler().fit_transform(df_pca)  


In [ ]:
pd.DataFrame(df_st, columns=df_pca.columns).head(5)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=11)
pca.fit(df_st)

In [ ]:
print(pca.explained_variance_)

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
print("variance by all the components=",sum(pca.explained_variance_ratio_*100))

###  **Q.4 Optimum value of K for K-means clustering using elbow method**

---

In [ ]:
df1=df.copy()
df1.drop('CUST_ID',inplace=True,axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
scale=StandardScaler().fit(df1)
df1=scale.transform(df1)

In [ ]:
df_scaled=pd.DataFrame(df1)
df_scaled.head()

In [ ]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters=i, init="k-means++", random_state = 11)
    kmeans.fit(df_scaled.iloc[:,3:])
    wcss.append(kmeans.inertia_)
plt.grid()
sns.lineplot(x=range(1,11), y=wcss, color="orange", marker ="8")
plt.xlabel("K-Value")
plt.xticks(np.arange(11.5))
plt.ylabel("WCSS")
plt.title("Elbow Graph")
plt.show()

***The optimal value of K in Elbow method is 3***

###  **Q.5 Optimum value of K for K-means clustering using Silhouette method**

---

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
n_clusters=[4,5,6,7,8]
for k in n_clusters:
          cluster=KMeans(n_clusters=k,random_state=10)
          predict=cluster.fit_predict(df_scaled)
          score=silhouette_score(df_scaled,predict,random_state=10)
          print("(for n_clusters={},silhouette_score  is {})".format(k,score))

 sillhout score is maximum at k = 8. So, we can take 8 clusters.